In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import keras

Using TensorFlow backend.


In [2]:
# Reading data

tweets_xlnet = pd.read_csv('../XLNet/test_with_pred_label_xlnet.csv', encoding="utf-8")
tweets_elmo = pd.read_csv('../ELMo/test_with_pred_label_elmo.csv', encoding="utf-8")
tweets_bert = pd.read_csv('../BERT/test_with_pred_label_bert.csv', encoding="utf-8")
tweets_true_labels = pd.read_csv('../Dataset/test_dataset.csv', encoding="utf-8")

#in this project BERT model drops first instance so we have to drop first instance of test data, elmo prediction and xlnet prediction.
tweets_true_labels.drop(tweets_true_labels.index[0], inplace=True)
tweets_xlnet.drop(tweets_xlnet.index[0], inplace=True)
tweets_elmo.drop(tweets_elmo.index[0], inplace=True)

xlnet_labels = list(tweets_xlnet['predicted_sentiment'])
elmo_labels = list(tweets_elmo['predicted_sentiment'])
bert_labels = list(tweets_bert['predicted_sentiment'])
true_labels = np.asarray(tweets_true_labels['sentiment'])

xlnet_labels_np = np.asanyarray(xlnet_labels)
elmo_labels_np = np.asanyarray(elmo_labels)
bert_labels_np = np.asanyarray(bert_labels)


print(tweets_true_labels.shape)

(12799, 4)


In [3]:
# Defining one-hot-ecoder and decoder

le = preprocessing.LabelEncoder()
le.fit(xlnet_labels)

def encode(labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [4]:
# Prioritizing models

xlnet_labels_one_hot = encode(xlnet_labels) * 0.55
elmo_labels_one_hot = encode(elmo_labels) * 0.6
bert_labels_one_hot = encode(bert_labels) * 0.65

In [5]:
# Ensembling

ensembled_result = xlnet_labels_one_hot + elmo_labels_one_hot + bert_labels_one_hot
temp = np.argmax(ensembled_result, axis=1)
ensembled_results = le.inverse_transform(temp)

In [6]:
# Calculate ensembled accuracy

ensembled_test_accuracy = (ensembled_results == true_labels).sum()/ensembled_results.shape[0]
print(f"Ensembled model has accuracy of {round(ensembled_test_accuracy*100,2)}% on test data")

Ensembled model has accuracy of 65.25% on test data


In [7]:
# Calculate enhancement the accuracy of each models after ensembling
import math

bert_enhancement_accuracy = ensembled_test_accuracy - ((bert_labels_np == true_labels).sum(axis=0))/ensembled_results.shape[0]
print(f"Bert model got enhanced {round(bert_enhancement_accuracy*100,2)}% after ensembling")

elmo_enhancement_accuracy = ensembled_test_accuracy - ((elmo_labels_np == true_labels).sum(axis=0))/ensembled_results.shape[0]
print(f"Elmo model got enhanced {round(elmo_enhancement_accuracy*100,2)}% after ensembling")

xlnet_enhancement_accuracy = ensembled_test_accuracy - ((xlnet_labels_np == true_labels).sum(axis=0))/ensembled_results.shape[0]
print(f"Xlnet model got enhanced {round(xlnet_enhancement_accuracy*100,2)}% after ensembling")

Bert model got enhanced -0.23% after ensembling
Elmo model got enhanced 5.02% after ensembling
Xlnet model got enhanced 11.23% after ensembling


In [8]:
# Calculate the similarity between result of seperate models and ensembled model

same_in_bert_and_ensembeled = (ensembled_results == bert_labels_np).sum()/ensembled_results.shape[0]
print(f"Bert model labels are {round(same_in_bert_and_ensembeled*100,2)}% similar with ensembled model lables")

same_in_elmo_and_ensembeled = (ensembled_results == elmo_labels_np).sum()/ensembled_results.shape[0]
print(f"Elmo model labels are {round(same_in_elmo_and_ensembeled*100,2)}% similar with ensembled model lables")

same_in_xlnet_and_ensembeled = (ensembled_results == xlnet_labels_np).sum()/ensembled_results.shape[0]
print(f"xlnet model labels are {round(same_in_xlnet_and_ensembeled*100,2)}% similar with ensembled model lables")

Bert model labels are 90.96% similar with ensembled model lables
Elmo model labels are 73.08% similar with ensembled model lables
xlnet model labels are 69.39% similar with ensembled model lables
